In [92]:
import numpy as np 
import pandas as pd 
import json
!cd C:/Users/gbao/Google 云端硬盘/BUDT758B/

In [94]:
import gc

path = 'C:/Users/gbao/Google 云端硬盘/BUDT758B/'
train_path = 'simplified-nq-train.jsonl'
test_path = 'simplified-nq-test.jsonl'
sample_submission_path = 'sample_submission.csv'

def read_data(path, sample = True, chunksize = 1000):
    if sample == True:
        df = []
        with open(path, 'rt') as reader:
            for i in range(chunksize):
                df.append(json.loads(reader.readline()))
        df = pd.DataFrame(df)
        print('Our sampled dataset have {} rows and {} columns'.format(df.shape[0], df.shape[1]))
    else:
        df = pd.read_json(path, orient = 'records', lines = True)
        print('Our dataset have {} rows and {} columns'.format(df.shape[0], df.shape[1]))
        gc.collect()
    return df

test = pd.read_json(path+test_path, orient = 'records', lines = True)


Our sampled dataset have 1000 rows and 6 columns


,document_text,long_answer_candidates,question_text,annotations,document_url,example_id
0,Email marketing - Wikipedia <H1> Email marketi...,"[{'start_token': 14, 'top_level': True, 'end_t...",which is the most common use of opt-in e-mail ...,"[{'yes_no_answer': 'NONE', 'long_answer': {'st...",https://en.wikipedia.org//w/index.php?title=Em...,5655493461695504401
1,The Mother ( How I Met Your Mother ) - wikiped...,"[{'start_token': 28, 'top_level': True, 'end_t...",how i.met your mother who is the mother,"[{'yes_no_answer': 'NONE', 'long_answer': {'st...",https://en.wikipedia.org//w/index.php?title=Th...,5328212470870865242
2,Human fertilization - wikipedia <H1> Human fer...,"[{'start_token': 14, 'top_level': True, 'end_t...",what type of fertilisation takes place in humans,"[{'yes_no_answer': 'NONE', 'long_answer': {'st...",https://en.wikipedia.org//w/index.php?title=Hu...,4435104480114867852
3,List of National Football League career quarte...,"[{'start_token': 28, 'top_level': True, 'end_t...",who had the most wins in the nfl,"[{'yes_no_answer': 'NONE', 'long_answer': {'st...",https://en.wikipedia.org//w/index.php?title=Li...,5289242154789678439
4,Roanoke Colony - wikipedia <H1> Roanoke Colony...,"[{'start_token': 32, 'top_level': True, 'end_t...",what happened to the lost settlement of roanoke,"[{'yes_no_answer': 'NONE', 'long_answer': {'st...",https://en.wikipedia.org//w/index.php?title=Ro...,5489863933082811018


In [102]:
example_id = []
ls = []
le = []
ss = []
se = []
with open("C:/Users/gbao/Downloads/predictions.json") as f:
    data = json.load(f)
for i in data.get('predictions'):
    example_id.append(float(i.get('example_id')))
    long = i.get('long_answer')
    ls.append(long.get('start_token'))
    le.append(long.get('end_token'))
    short = i.get('short_answers')[0]
    ss.append(short.get('start_token'))
    se.append(short.get('end_token'))
df_pred = pd.DataFrame({'example_id':example_id,'long_start':ls,'long_end':le,'short_start':ss,'short_end':se})
pred_merged = pd.concat([df_pred,test],axis=1)

In [123]:
def token_to_string(s):
    str1 = "" 
    return (str1.join(s)) 

pred_merged["short_answer"] = pred_merged["document_text"]
pred_merged["long_answer"] = pred_merged["document_text"] 
for i in pred_merged.index:
    ls = int(pred_merged.loc[i,"long_start"])
    le = int(pred_merged.loc[i,"long_end"])
    ss = int(pred_merged.loc[i,"short_start"])
    se = int(pred_merged.loc[i,"short_end"])
    text = pred_merged.loc[i,"document_text"].split()
    pred_merged.loc[i,"long_answer"] = token_to_string(text[ls:le])
    pred_merged.loc[i,"short_answer"] = token_to_string(text[ss:se])
pred_ans = pred_merged[['question_text','document_text','short_answer','long_answer']]
pred_ans.to_csv('C:/Users/gbao/Downloads/QA_predict.csv')

In [122]:
pred_ans

,question_text,document_text,short_answer,long_answer
0,who is the south african high commissioner in ...,"High Commission of South Africa , London - wik...",0.1269°W﻿/51.5082,"<Table><Tr><Th_colspan=""2"">HighCommissionofSou..."
1,the office episode when they sing to michael,Michael 's Last Dundies - wikipedia <H1> Micha...,.MichaelsoonlearnsthatDeangelohasaterribleprob...,<P>Theseriesdepictstheeverydaylivesofofficeemp...
2,what is the main idea of the cross of gold speech,Cross of gold speech - wikipedia <H1> Cross of...,'saddresshelpedcatapult,<P>TheCrossofGoldspeechwasdeliveredbyWilliamJe...
3,when was i want to sing in opera written,Wilkie Bard - wikipedia <H1> Wilkie Bard </H1>...,information</Th></Tr><Tr><Th>Birthname</Th><Td...,"<Table><Tr><Th_colspan=""2"">WilkieBard</Th></Tr..."
4,who does the voices in ice age collision course,Ice Age : Collision Course - Wikipedia <H1> Ic...,"CollisionCourse</Th></Tr><Tr><Td_colspan=""2"">T...","<Table><Tr><Th_colspan=""2"">IceAge:CollisionCou..."
...,...,...,...,...
341,how many season of last man standing are there,Last Man Standing ( U.S. TV series ) - wikiped...,Shipley</Li><Li>MattBerry</Li><Li>TimDoyle</Li...,"<Table><Tr><Th_colspan=""2"">LastManStanding</Th..."
342,where does the black forest cake come from,Black Forest gâteau - wikipedia <H1> Black For...,",thatgivesthedessertitsflavour.Cherries,",<P>ThedessertisnotdirectlynamedaftertheBlackFo...
343,india won olympic gold medal in hockey 1948,India at the 1948 Summer Olympics - wikipedia ...,Time(BST).IndianStandardTime(,<P>Note:AlltimesmentionedonthispageareinBritis...
344,most viewed youtube music video in 24 hours re...,List of most - viewed online videos in the fir...,"31.4</Td><Td>November5,2018</Td><Td></Td></Tr>...",<Table><Tr><Th>Rank</Th><Th>Title</Th><Th>Uplo...
